### Connect and use Hopsworks Feature Store from Databricks

This notebook assumes that you have alreadly followed the instructions in the `Databricks-FeatureStore-Setup` notebook. In particular you should have already run the `setup_databricks` method and configured the cluster with the additional Spark Settings and Init Script.

Before being able to read/write into the Hopsworks Feature Store we need to connect to it. The connection parameters are the same of the `setup_databricks` method. As before, if you don't have access to the Secrets Manager or Parameter Store, you have the option of writing the API key on a file and pass the `api_key_file` parameter.

In [2]:
from  hops import featurestore

featurestore.connect(host="instance.aws.hopsworks.ai", project_name="demo_featurestore_admin000", region_name="us-west-2", secrets_store="secretsmanager")

Once the connection has been established correctly we can start using the Hops Python API to interact with the Hopsworks Feature Store. You can find the API documentation at http://hops-py.logicalclocks.com. You can also run all the PySpark example you can find in our `hops-examples` Github repository: https://github.com/logicalclocks/hops-examples/tree/master/notebooks/featurestore - Just make sure you prepend the `featurestore.connect` method, before running any of the cells.

In the next cell we are going to demonstrate how you can use the Hops Python API to read an existing feature group and create a new one.

In [4]:
attendances_features = featurestore.get_featuregroup("attendances_features")

Running sql: use demo_featurestore_fabio000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM attendances_features_1 against offline feature store

In [5]:
attendances_features.show(5)

+-------+------------------+--------------+
team_id|average_attendance|sum_attendance|
+-------+------------------+--------------+
 6| 19595.973| 391919.47|
 16| 6462.462| 129249.24|
 20| 7226.672| 144533.44|
 40| 3189.8455| 63796.91|
 9| 9405.213| 188104.27|
+-------+------------------+--------------+
only showing top 5 rows

For the sake of the example, let's add another column to the dataframe which will become the content for a new feature group.

In [7]:
from pyspark.sql.functions import rand
new_fg = attendances_features.withColumn("avg_age", rand() * 100)

Using the Hops Python API we can now write into the Hopsworks Feature Store the new Feature Group.

In [9]:
featurestore.create_featuregroup(
    new_fg,
    "attendances_features_databricks",
    description="Feature Group containing average attendances features created from Databricks",
    descriptive_statistics=True,
    feature_correlation=True,
    feature_histograms=True,
    cluster_analysis=True
)

computing descriptive statistics for : attendances_features_databricks, version: 1
computing feature correlation for: attendances_features_databricks, version: 1
computing feature histograms for: attendances_features_databricks, version: 1
computing cluster analysis for: attendances_features_databricks, version: 1
/databricks/spark/python/pyspark/sql/dataframe.py:2160: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)
Registering feature metadata...
Registering feature metadata... [COMPLETE]
Writing feature data to offline feature group (Hive)...
Running sql: use demo_featurestore_fabio000_featurestore against offline feature store
Writing feature data to offline feature group (Hive)... [COMPLETE]
Feature group created successfully